# Pydantic Fundamentals

## validate_call

In [5]:
# Try without and with python type hints
def soma(a, b):
    return a + b

print(soma(1, 2))

def soma(a: int, b: int) -> int:
    return a + b

print(soma(2, 2))
print(soma("som", "as"))

from pydantic import validate_call

@validate_call
def soma(a: int, b: int) -> int:
    return a + b

print(soma(2, 2))
print(soma(1, "as"))

3
4
somas
4


ValidationError: 1 validation error for soma
1
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='as', input_type=str]
    For further information visit https://errors.pydantic.dev/2.1/v/int_parsing

In [9]:
from typing import Union
from pydantic import validate_call

def soma(a: int, b: Union[int, str]):
    return a + b

print(soma(1, 2))
print(soma('1', '3'))

@validate_call
def soma(a: Union[int, str], b: Union[int, str]):
    return a + b

print(soma(1, 2))
print(soma('1', '3'))

3
13
3
13


## Data Classes

In [22]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    age: int

# This class dont have any validation
p1 = Person(name='John', age='42')
print(p1)

from pydantic.dataclasses import dataclass as pydantic_dataclass

@pydantic_dataclass
class PersonPydantic:
    name: str
    age: int


# This class have validation and will change str to int
p2 = PersonPydantic(name='John', age='42')
print(p2)


# Can use strictInt to only receive int and DONT change str to int
from pydantic import StrictInt

@pydantic_dataclass
class PersonPydanticStrict:
    name: str
    age: StrictInt

p3 = PersonPydanticStrict(name='John', age='42')

Person(name='John', age='42')
PersonPydantic(name='John', age=42)


ValidationError: 1 validation error for PersonPydanticStrict
age
  Input should be a valid integer [type=int_type, input_value='42', input_type=str]
    For further information visit https://errors.pydantic.dev/2.1/v/int_type

## Base Models

In [1]:
from pydantic import BaseModel

class Register(BaseModel):
    name: str
    age: int
    email: str
    password: str
    is_active: bool = True


r = Register(name='John', age=20, email="admin@admin.com", password="123456", is_active=True)